In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

import joblib

import re

In [2]:
file_path = "병합_청약매물_목록_정보.csv"

df = pd.read_csv(file_path, encoding='cp949')

In [ ]:
def filter_unnecessary_rows(df):
    # 민간주택만 필터링 (공공주택은 가점제 없음)
    df = df[df['주택상세구분코드명'] != '국민'].reset_index(drop=True)

    # 서울, 경기, 인천 지역만 필터링
    df = df[df['공급지역명'].isin(['서울', '경기', '인천'])].reset_index(drop=True)
    
    return df

In [59]:
df = filter_unnecessary_rows(df)
df

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점
0,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,1,1,해당지역,13,(△11),0.0,0.00,0.0
1,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,1,2,기타지역,147,13.36,0.0,0.00,0.0
2,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,2,1,해당지역,0,NaN,NaN,NaN,NaN
3,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,2,2,기타지역,0,NaN,NaN,NaN,NaN
4,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9347B,13,1,1,해당지역,27,2.08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.9100,43,2,2,기타지역,0,NaN,NaN,NaN,NaN
18856,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,1,1,해당지역,16,(△6),23.0,31.36,51.0
18857,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,1,2,기타지역,138,23.00,23.0,31.36,51.0
18858,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,2,1,해당지역,0,NaN,NaN,NaN,NaN


### 불필요 컬럼 제거

In [60]:
def filter_unnecessary_columns(df):
    # 불필요 칼럼 확인
    unnecessary_columns = []
    
    # Case 1) 칼럼 값이 하나라면 불필요 칼럼
    for column in df.columns:
        if len(df[column].unique()) == 1:
            unnecessary_columns.append(column)
    
    # Case 2) 홈페이지 주소, 문의처 칼럼
    for column in df.columns:
        if column in ['홈페이지주소', '모집공고홈페이지주소', '문의처']:
            unnecessary_columns.append(column)
    
    # Case 3) 특별공급 관련 칼럼
    for column in df.columns:
        if '특별공급' in column:
            unnecessary_columns.append(column)
    
    # Case 4) 청약접수종료일을 제외한 나머지 일자 칼럼
    for column in df.columns:
        if column != '청약접수종료일':
            if ('시작일' in column or '종료일' in column):
                unnecessary_columns.append(column)
    
    # Case 5) 그 외 기타 칼럼
    unnecessary_columns.extend(["주택관리번호", "모델번호", '주택관리번호', '공급위치우편번호', '모집공고일', '당첨자발표일', '입주예정월'])
    
    # 시공사랑 시행사는 불필요할것같아서 추가로 제거할게요
    unnecessary_columns.extend(['건설업체명_시공사', '사업주체명_시행사'])
    
    # 중복된 칼럼 제거
    unnecessary_columns = list(set(unnecessary_columns))
    
    # 불필요한 칼럼 삭제
    df = df.drop(columns=unnecessary_columns)
    
    return df


In [61]:
# 사용 예시
df = filter_unnecessary_columns(df)
df


,공고번호,주택명,공급지역코드,공급지역명,공급위치,공급규모,청약접수종료일,투기과열지구,조정대상지역,분양가상한제,...,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점
0,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,075.9493A,24,1,1,해당지역,13,(△11),0.0,0.00,0.0
1,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,075.9493A,24,1,2,기타지역,147,13.36,0.0,0.00,0.0
2,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,075.9493A,24,2,1,해당지역,0,NaN,NaN,NaN,NaN
3,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,075.9493A,24,2,2,기타지역,0,NaN,NaN,NaN,NaN
4,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,075.9347B,13,1,1,해당지역,27,2.08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,074.9100,43,2,2,기타지역,0,NaN,NaN,NaN,NaN
18856,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,074.8900,22,1,1,해당지역,16,(△6),23.0,31.36,51.0
18857,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,074.8900,22,1,2,기타지역,138,23.00,23.0,31.36,51.0
18858,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,074.8900,22,2,1,해당지역,0,NaN,NaN,NaN,NaN


### 전처리

In [62]:
def split_housing_type(df):
    if '주택형' in df.columns:
        # '.'을 기준으로 왼쪽 숫자 뽑아내서 전용면적 컬럼 생성
        df['전용면적'] = df['주택형'].apply(lambda x: int(x.split('.')[0].lstrip('0')))
        
        # '.'을 기준으로 오른쪽 숫자+영어 뽑아내서 평면유형 컬럼 생성
        df['평면유형'] = df['주택형'].apply(lambda x: x.split('.')[1] if '.' in x else x)
        
        # Drop the original '주택형' column
        #df = df.drop(columns=['주택형'])
    
    return df

In [63]:
# 예시
df = split_housing_type(df)
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치,공급규모,청약접수종료일,투기과열지구,조정대상지역,분양가상한제,...,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점,전용면적,평면유형
0,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,1,해당지역,13,(△11),0.0,0.00,0.0,75,9493A
1,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,2,기타지역,147,13.36,0.0,0.00,0.0,75,9493A
2,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,1,해당지역,0,NaN,NaN,NaN,NaN,75,9493A
3,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,2,기타지역,0,NaN,NaN,NaN,NaN,75,9493A
4,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,1,해당지역,27,2.08,NaN,NaN,NaN,75,9347B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,2,기타지역,0,NaN,NaN,NaN,NaN,74,9100
18856,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,1,해당지역,16,(△6),23.0,31.36,51.0,74,8900
18857,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,2,기타지역,138,23.00,23.0,31.36,51.0,74,8900
18858,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,1,해당지역,0,NaN,NaN,NaN,NaN,74,8900


경쟁률 미달 처리

In [64]:
def preprocessing_applicant_rate(df):

    def process_rate(row):
        # 미달인 경우 경쟁률 처리
        if '△' in str(row['경쟁률']):
            pattern = '[^0-9]'
            shortage = int(re.sub(pattern, '', str(row['경쟁률'])))
            
            if row['접수건수'] == 0:
                rate = 0
            else:
                rate = round((row['공급세대수'] - shortage) / row['공급세대수'], 2)
        # 쉼표 제거 후 float으로 변환
        else:
            rate = float(str(row['경쟁률']).replace(',', ''))
        
        # 미달여부 판단
        shortage_status = 'Y' if rate < 1 else 'N'
        
        return pd.Series({'경쟁률': rate, '미달여부': shortage_status})

    df[['경쟁률', '미달여부']] = df.apply(process_rate, axis=1)
    
    return df


In [65]:
# 사용예시
df = preprocessing_applicant_rate(df)
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치,공급규모,청약접수종료일,투기과열지구,조정대상지역,분양가상한제,...,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점,전용면적,평면유형,미달여부
0,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,13,0.54,0.0,0.00,0.0,75,9493A,Y
1,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,기타지역,147,13.36,0.0,0.00,0.0,75,9493A,N
2,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,0,NaN,NaN,NaN,NaN,75,9493A,N
3,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,기타지역,0,NaN,NaN,NaN,NaN,75,9493A,N
4,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,27,2.08,NaN,NaN,NaN,75,9347B,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,기타지역,0,NaN,NaN,NaN,NaN,74,9100,N
18856,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,해당지역,16,0.73,23.0,31.36,51.0,74,8900,Y
18857,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,기타지역,138,23.00,23.0,31.36,51.0,74,8900,N
18858,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,해당지역,0,NaN,NaN,NaN,NaN,74,8900,N


경쟁률 NaN값 처리

In [66]:
def fill_nan_with_zero(df):
    df["경쟁률"] = df["경쟁률"].fillna(0)
    return df

In [67]:
# 예시
df = fill_nan_with_zero(df)
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치,공급규모,청약접수종료일,투기과열지구,조정대상지역,분양가상한제,...,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점,전용면적,평면유형,미달여부
0,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,13,0.54,0.0,0.00,0.0,75,9493A,Y
1,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,기타지역,147,13.36,0.0,0.00,0.0,75,9493A,N
2,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,0,0.00,NaN,NaN,NaN,75,9493A,N
3,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,2,기타지역,0,0.00,NaN,NaN,NaN,75,9493A,N
4,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,1,해당지역,27,2.08,NaN,NaN,NaN,75,9347B,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,기타지역,0,0.00,NaN,NaN,NaN,74,9100,N
18856,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,해당지역,16,0.73,23.0,31.36,51.0,74,8900,Y
18857,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,2,기타지역,138,23.00,23.0,31.36,51.0,74,8900,N
18858,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,1,해당지역,0,0.00,NaN,NaN,NaN,74,8900,N


공급금액 데이터 병합

In [68]:
def add_estate_price(df):
    # 공급금액 데이터 불러오기
    df_estate_price = pd.read_csv("청약매물_공급금액 (서울, 경기, 인천).csv", encoding='cp949')
    df_estate_price = df_estate_price[['공고번호', '주택형', '공급금액(최고가 기준)']]

    df_estate_price.drop_duplicates(subset=['공고번호', '주택형'], keep='first', inplace=True)
    
    # 원본 데이터에 공급금액 칼럼 추가하기
    df = pd.merge(df, df_estate_price, on=['공고번호', '주택형'], how='left')

    return df

In [69]:
df = add_estate_price(df)
df

,공고번호,주택명,공급지역코드,공급지역명,공급위치,공급규모,청약접수종료일,투기과열지구,조정대상지역,분양가상한제,...,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점,전용면적,평면유형,미달여부,공급금액(최고가 기준)
0,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,해당지역,13,0.54,0.0,0.00,0.0,75,9493A,Y,664170000.0
1,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,기타지역,147,13.36,0.0,0.00,0.0,75,9493A,N,664170000.0
2,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,해당지역,0,0.00,NaN,NaN,NaN,75,9493A,N,664170000.0
3,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,기타지역,0,0.00,NaN,NaN,NaN,75,9493A,N,664170000.0
4,2024000706,의왕 월암지구 대방 디에트르 레이크파크,410,경기,경기도 의왕시 월암동 공공주택지구 B2블록(경기도 의왕시 월암동 219-3일원),702,2025-01-02,N,N,Y,...,해당지역,27,2.08,NaN,NaN,NaN,75,9347B,N,591150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18855,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,기타지역,0,0.00,NaN,NaN,NaN,74,9100,N,NaN
18856,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,해당지역,16,0.73,23.0,31.36,51.0,74,8900,Y,NaN
18857,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,기타지역,138,23.00,23.0,31.36,51.0,74,8900,N,NaN
18858,2020000009,의왕 오전 동아루미체,410,경기,경기도 의왕시 전주남이길 23-26 (오전동),65,2020-02-19,N,N,N,...,해당지역,0,0.00,NaN,NaN,NaN,74,8900,N,NaN


### 데이터 파이프 라인 준비

1. 컬럼 전처리

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

# FunctionTransformer로 함수 변환
filter_transformer = FunctionTransformer(filter_unnecessary_columns)
split_transformer = FunctionTransformer(split_housing_type)
rate_transformer = FunctionTransformer(preprocessing_applicant_rate)
nan_transformer = FunctionTransformer(fill_nan_with_zero)
price_transformer = FunctionTransformer(add_estate_price)


# 파이프라인 구성
preprocessing_pipeline = Pipeline([
    ("filter", filter_transformer),
    ("split", split_transformer),
    ("rate", rate_transformer),
    ("nan", nan_transformer),
    ("price", price_transformer)
])
